In [1]:
import torch

import lightning as L
import matplotlib.pyplot as plt

from PIL import Image
from diffusers import DDPMScheduler
from datasets import load_dataset
from torchvision import transforms
from diffusers import UNet2DModel
from diffusers.optimization import get_cosine_schedule_with_warmup

from TL.generation import Tempering

In [2]:
image_size = 32
train_batch_size = 16
T = 100
num_epochs = T
learning_rate = 1e-4
lr_warmup_steps = 100

In [3]:
dataset = load_dataset(
    "huggan/smithsonian_butterflies_subset", 
    split="train", 
    cache_dir="./cache"
)

preprocess = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}

dataset.set_transform(transform)

sample_image = dataset[0]["images"].unsqueeze(0)

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
train_dataloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=train_batch_size, 
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

In [5]:
model = UNet2DModel(
    sample_size=image_size,  # the target image resolution
    in_channels=3,  # the number of input channels, 3 for RGB images
    out_channels=3,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(32, 32, 64, 64, 128, 128),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 7126275


In [6]:
model.sample_size


32

In [7]:
noise_scheduler = DDPMScheduler()

In [8]:
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr=learning_rate
)

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [9]:
tlmodel = Tempering(
    model=model,
    noise_scheduler=noise_scheduler,
    T=100, 
    num_training_samples=len(dataset),
    num_mc_steps=100,
    burn_in_fraction=0.5,
    zeta=10.0,
    mc_subset_ratio=0.5,
    random_walk_step_size=0.0,
    prediction_type="epsilon"
)

In [11]:
tlmodel.timesteps.device

device(type='cpu')

In [12]:
import torchvision
images = tlmodel.generate_samples(10)
images = (images / 2 + 0.5).clamp(0, 1).squeeze()
grid = torchvision.utils.make_grid(images, nrow=5)
images = (grid.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
images = Image.fromarray(images)    
images.save("images.png")

In [ ]:
tlmodel.device

In [ ]:
trainer = L.Trainer(
    max_epochs=num_epochs,
    accelerator="gpu",
    precision=32,
    fast_dev_run=False
)
trainer.fit(tlmodel, train_dataloader)

In [19]:
pure_noise = torch.randn_like(sample_image)
denoised_image = tlmodel.model(pure_noise, 100).sample

In [14]:
image = (denoised_image / 2 + 0.5).clamp(0, 1).squeeze()
image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
image = Image.fromarray(image)
# save image
image.save("denoised_image.png")

In [ ]:
Image.fromarray(((denoised_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])